In [1]:
import matplotlib
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
import scipy.interpolate
import mne
import scipy
import statistics
from pywt import wavedec
from scipy import signal
import matplotlib.colors as mcolors
import random
import seaborn as sns
import os
from scipy import stats
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
from Definicions import upperchanel
from Definicions import group_inf
#from Definicions import mediumchanel
from Definicions import opteciogrups
from Definicions import find_nearest
from Definicions import grabt
from Definicions import epoch_return
from Import_all import Import_Patients #to create a list of the patiens with the EO-EC data and 12 chanels
from BandPower_Epoch_1c import BanPoer_Epoch
from BandPower_Epoch_1c import bandpower

In [3]:
###READ FILES IN THE FOLDER AND SAVE THEM
basePATH = 'Files/Preprocessed/'
#Save the list of names separated EO vs EC
listPATHEC = []
listPATHEO = []
#Save only the set files given that in the upload they automaticly search for the .fdt
for root, dirs, files in os.walk(str(basePATH)):
    for filename in files:
        if '.set' in filename:
            if 'EC' in filename:
                listPATHEC.append(filename)
            if 'EO' in filename:
                listPATHEO.append(filename)
#Save again but sorted by the name that way we know they are in order
sorted_list_EO = sorted(listPATHEO)
sorted_list_EC = sorted(listPATHEC)


In [4]:
EO_EC_Pacients = []
for i in sorted_list_EO:
    #Load the doc
    x=mne.io.read_raw_eeglab(basePATH+i, preload=True, verbose=True)
    #GET DATA
    data = x._data
    #GET CHANELS
    chanles_names = x.ch_names

    #Select the same but with EC
    EC_name = [sorted_list_EC.index(e) for e in sorted_list_EC if i[:-6] in e]
    EC_name_PATH = sorted_list_EC[EC_name[0]]
    #LOAD EC
    x2=mne.io.read_raw_eeglab(basePATH+EC_name_PATH, preload=True, verbose=True)
    #GET DATA
    data2 = x2._data

    #save
    EO_EC_P = []
    EO_EC_P.append(data)
    EO_EC_P.append(data2)
    EO_EC_Pacients.append(EO_EC_P)

564 secs...
Reading Files/Preprocessed/sub-010196_EO.fdt
Reading 0 ... 117057  =      0.000 ...   468.228 secs...
Reading Files/Preprocessed/sub-010196_EC.fdt
Reading 0 ... 119674  =      0.000 ...   478.696 secs...
Reading Files/Preprocessed/sub-010197_EO.fdt
Reading 0 ... 117064  =      0.000 ...   468.256 secs...
Reading Files/Preprocessed/sub-010197_EC.fdt
Reading 0 ... 118379  =      0.000 ...   473.516 secs...
Reading Files/Preprocessed/sub-010199_EO.fdt
Reading 0 ... 118643  =      0.000 ...   474.572 secs...
Reading Files/Preprocessed/sub-010199_EC.fdt
Reading 0 ... 118755  =      0.000 ...   475.020 secs...
Reading Files/Preprocessed/sub-010200_EO.fdt
Reading 0 ... 115768  =      0.000 ...   463.072 secs...
Reading Files/Preprocessed/sub-010200_EC.fdt
Reading 0 ... 108500  =      0.000 ...   434.000 secs...
Reading Files/Preprocessed/sub-010201_EO.fdt
Reading 0 ... 117660  =      0.000 ...   470.640 secs...
Reading Files/Preprocessed/sub-010201_EC.fdt
Reading 0 ... 117788  =  

In [8]:
len(EO_EC_Pacients[1][0][0])

120000

In [38]:
pacient_beta_EO = []

pacient_alpha_EO = []

pacient_gama_EO = []

pacient_theta_EO = []

pacient_delta_EO = []

for pacient in EO_EC_Pacients:
    EO_Epochs = epoch_return(pacient[1])
    chanels_betta = []
    chanels_gama = []
    chanels_alpha = []
    chanels_theta = []
    chanels_delta = []
    for chane in EO_Epochs:
        for epoch in chane:
            f_beta = bandpower(epoch, 250,[30, 250], window_sec=1, relative=True)
            chanels_betta.append(f_beta)
            #gamma
            f_gama = bandpower(epoch, 250, [32, 100], window_sec=1, relative=True)
            chanels_gama.append(f_gama)
            #alpha
            f_alpha = bandpower(epoch, 250, [9, 13], window_sec=1, relative=True)
            chanels_alpha.append(f_alpha)
            #theta
            f_theta = bandpower(epoch, 250, [4, 8], window_sec=1, relative=True)
            chanels_theta.append(f_theta)
            #delta
            f_delta = bandpower(epoch, 250, [0.1, 4], window_sec=1, relative=True)
            chanels_delta.append(f_delta)
            
    #for chanels

    mean_x = statistics.median(chanels_betta)
    pacient_beta_EO.append(mean_x)

    mean_x = statistics.median(chanels_gama)
    pacient_alpha_EO.append(mean_x)

    mean_x = statistics.median(chanels_alpha)
    pacient_gama_EO.append(mean_x)

    mean_x = statistics.median(chanels_theta)
    pacient_theta_EO.append(mean_x)

    mean_x = statistics.median(chanels_delta)
    pacient_delta_EO.append(mean_x)


In [39]:
len(pacient_beta_EO)

186

In [13]:
'''import pickle

with open("pacient_beta_EO.txt", "wb") as fp:   #Pickling
    pickle.dump(pacient_beta_EO, fp)


with open("pacient_alpha_EO.txt", "wb") as fp:   #Pickling
    pickle.dump(pacient_alpha_EO, fp)


with open("pacient_gama_EO.txt", "wb") as fp:   #Pickling
    pickle.dump(pacient_gama_EO, fp)

with open("pacient_theta_EO.txt", "wb") as fp:   #Pickling
    pickle.dump(pacient_theta_EO, fp)


with open("pacient_delta_EO.txt", "wb") as fp:   #Pickling
    pickle.dump(pacient_delta_EO, fp)'''


In [40]:
import csv
from itertools import zip_longest
list0 = keys
list1 = pacient_beta_EO
list2 = pacient_delta_EO
list3 = pacient_gama_EO
list4 = pacient_theta_EO
list5 = pacient_alpha_EO
d = [list0, list1, list2, list3, list4, list5]
export_data = zip_longest(*d, fillvalue = '')
with open('pacient_bandpower_5.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("Num_pacient", "Betta", "Delta", "Gamma", "Theta", "Alpha"))
      wr.writerows(export_data)
myfile.close()

In [37]:
pacient_beta_EO[2]

0.024740205891482705